In [29]:
%load_ext autoreload
%autoreload 2

from utils import load_config, set_environment_variables, num_tokens_from_messages, get_git_files, get_data_files
import sys
import os
config       = load_config('config.yaml')
set_environment_variables(config)
ACCESS_TOKEN = os.environ['GIT_ACCESS_TOKEN']
sys.path.append("/database")

In [30]:
from database import get_documentation_by_url, put_new_repository_documentation, get_file_documentation, delete_documentation_by_url, get_status, get_status_by_file, external_json_to_repo_overview_output
from repo_processor import process_file, parallel_process_files, download_and_process_repo_url, get_repo_overview
from OverviewChain import OverviewParser, ConfluenceOverviewChain
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.document_loaders import GithubFileLoader
from langchain.callbacks import get_openai_callback
from ConfluenceChain import ConfluenceChain, Parser
from pymongo import MongoClient
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import nest_asyncio
import shutil
nest_asyncio.apply()


In [31]:
repo_url              = "https://github.com/Adarsh9616/Electricity_Billing_System/"
# repo_url              = "https://github.com/thuml/Time-Series-Library/"
supported_languages   = ['python', 'java', 'javascript']

# data_dict             = download_and_process_repo_url(repo_url, supported_languages)

In [31]:
# import boto3
# sqs       = boto3.client('sqs', 'us-east-1')
# queue_url = 'https://sqs.us-east-1.amazonaws.com/018192622412/doc-gen'
# response  = sqs.send_message(
#     QueueUrl=queue_url,
#     MessageBody=str("https://github.com/Adarsh9616/Electricity_Billing_System/")
# )

# """ or """

# https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url=https://github.com/Adarsh9616/Electricity_Billing_System/

In [30]:
# response = sqs.receive_message(
#     QueueUrl=queue_url,
#     MaxNumberOfMessages=1,
#     WaitTimeSeconds=1 # Long polling
# )

# messages = response.get('Messages', [])
# print(messages)
# while len(messages) != 0:
#     response = sqs.receive_message(
#         QueueUrl=queue_url,
#         MaxNumberOfMessages=1,
#         WaitTimeSeconds=1 # Long polling
#     )

#     messages = response.get('Messages', [])
#     print(messages)

[{'MessageId': '907db49d-4915-4b94-b643-bed34220873c', 'ReceiptHandle': 'AQEBPOEwO9zDxF4fb+4Av8xejsokvQWSp2x6ceBivpdwfZI9SReJic4X/GbJEpqBwYK+FmvRn1bYEEbCQwWdwnWjJrG3vxiQwyPNk0JqenT9mjMJm1YXqWQ+NeHvQ/GvK3O9PGpxisrC/TigpvIpntgImzFbA+u/R1puRfGf9kzgdLNalyJkOJanM8Dc46QfT37KB+Qch4J+BY97QklQmo//Xs9sWbfUivJxbarHCMi0isYsRa9L0Vjh2wTOYDIcBw1/tLDOySfWejeH7knNVUPdwsHMXpwtyK3mUm77jqJ2KBvqrfOHMJ73RKsjn0tn6mTc+LjFHPac4YKj2otWQmZDtTXER+PF+dKgey/RHojEKaKEnnm/LVkIn+PmstAbd2ER', 'MD5OfBody': '5e8c8de0e3a0c052ec15a8ed582f785c', 'Body': 'https://github.com/Adarsh9616/Electricity_Billing_System/'}]
[]


In [32]:
status_url = get_status(repo_url)
# get_status_by_file(repo_url, file_name)
status_url

{'repository_url': 'https://github.com/Adarsh9616/Electricity_Billing_System/',
 'overall_status': 'Completed',
 'file_level_status': {'src/conn.java': 'Completed',
  'src/Project.java': 'Completed',
  'src/pay.bill.java': 'Completed',
  'src/login.java': 'Completed',
  'src/customer.details.java': 'Completed',
  'src/splash.java': 'Completed',
  'src/calculate.bill.java': 'Completed',
  'src/new.customer.java': 'Completed',
  'src/generate.bill.java': 'Completed',
  'src/LastBill.java': 'Completed',
  'repo.overview.data': 'Completed'}}

In [33]:
# edit database_json_to_respsitory_confluence_output
# edit add_project_overview_to_repository

In [34]:
docs = get_documentation_by_url(repo_url)
docs.files

Now the file path is src/conn.java
Now the file path is src/splash.java
Now the file path is src/Project.java
Now the file path is src/login.java
Now the file path is src/calculate.bill.java
Now the file path is src/new.customer.java
Now the file path is src/customer.details.java
Now the file path is src/generate.bill.java
Now the file path is src/pay.bill.java
Now the file path is src/LastBill.java
Now the file path is src/conn.java


{'src/conn.java': FileConfluenceOutput(file_path='src/conn.java', overall_summary="This file defines a Java class named 'conn' that is responsible for establishing a connection to a MySQL database. It primarily serves the purpose of initializing the database connection and creating a statement object that can be used for executing SQL queries.", packages={'java.sql': PackageDetail(usage='Used for database connectivity and execution of SQL statements.', description='This package provides the API for accessing and processing data stored in a data source (usually a relational database) using the Java programming language. It includes classes and interfaces for handling database connections, executing SQL queries, and managing the resulting data.')}, functions={'conn': FunctionDetail(name='conn', description="Constructor of the 'conn' class that initializes the database connection and statement object.", class_declaration='public class conn { Connection c; Statement s; public conn() {...} 

In [6]:
# delete_documentation_by_url(repo_url)

In [35]:
name_of_repo = repo_url.split("/")[-2]
res          = get_repo_overview(name_of_repo, str(docs.files))

2024-04-22 01:28:34,069 - INFO - Total tokens for repo overview data: 4723


KeyboardInterrupt: 

In [27]:
# repo_url            = "https://github.com/Adarsh9616/Electricity_Billing_System/"
repo_url            = "https://github.com/dbhatia00/documentation-generation/"
url_to_process_repo = f"https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url={repo_url}"

import requests
response            = requests.get(url_to_process_repo)


In [2]:
response.json()

'Repository URL successfully enqueued: https://github.com/dbhatia00/documentation-generation/'

In [24]:
from langchain_community.document_loaders import GitLoader

clone_url = "https://github.com/dbhatia00/documentation-generation/"
repo_path = "/tmp/documentation-generation"

repo_name_with_owner  = repo_url.split("github.com/")[1]
repo_name             = repo_name_with_owner.split("/")[1]
repo_path             = f"/tmp/{repo_name}"
import os
import shutil
if os.path.exists(repo_path):
    shutil.rmtree(repo_path)
loader = None
files  = None





In [25]:
for br in ['main', 'master']:
    try:
        print(br)
        loader = GitLoader(
            clone_url=clone_url,
            repo_path=repo_path,
            branch=br,
            # file_filter=lambda file_path: file_path.endswith(".py") #or file_path.endswith(".java") or file_path.endswith(".js"),
        )
        files = loader.load()
        
        # logger.info(f"Files loaded for branch {br}")
        break
    except Exception as e:
        # logger.error(f"Error loading git files: {e} for branch {br}")
        pass

print(loader)
print(files)
if not loader or not files:
    # logger.error("Error loading git files: Loader not initialized")
    raise ValueError("Error loading git files: Loader not initialized")

main
[Document(page_content='env\n.vscode\nflask-be/token_server.json\nreact-fe/src/token_client.json\n**.yaml\n**example_data**', metadata={'source': '.gitignore', 'file_path': '.gitignore', 'file_name': '.gitignore', 'file_type': ''}), Document(page_content='# documentation-generation\n\n## Overview\nDocumentation is difficult to write and keep up to date.  Tools exist to generate rudimentary JavaDoc for Java programs, but the result isn’t particularly helpful as it just gives parameter names and return types from the signature and no useful information.  The goal of this project is to provide a more reasonable initial documentation source using LLMs and other tools.  There have been a number of research efforts along this line that could be a good starting point.\n\n## Initial Implementation Goals\n\n### Description\nA web app that, when provided a github link, generates a confluence page describing what the app does based on classes/methods within. This process will work on most ma

In [26]:
loader = GitLoader(
    clone_url=clone_url,
    repo_path=repo_path,
    branch='main',
    # file_filter=lambda file_path: file_path.endswith(".py") #or file_path.endswith(".java") or file_path.endswith(".js"),
)
files = loader.load()